# test

In [ ]:
from textblob import TextBlob
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from operator import itemgetter
from wordcloud import WordCloud
from pathlib import Path
# handling images
# https://stackoverflow.com/questions/35286540/display-an-image-with-python
import matplotlib.image as mpimg
import preprocessor as p
from pprint import pprint

In [ ]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
import datetime
import csv

In [ ]:
import keys

In [ ]:
auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)

In [ ]:
auth.set_access_token(keys.access_token,keys.access_token_secret)

In [ ]:
# Rate limiting

In [ ]:
api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [ ]:
# Session 1

In [ ]:
import tweepy

#https://stackoverflow.com/questions/30721567/limit-tweepy-stream-to-a-specific-number
#https://www.storybench.org/how-to-collect-tweets-from-the-twitter-streaming-api-using-python/

# authorization tokens
#consumer_key = "[insert your key here]"
#consumer_secret = "[insert your secret here]"
#access_key = "[insert your key here]"
#access_secret = "[insert your secret here]"

# StreamListener class inherits from tweepy.StreamListener and overrides on_status/on_error methods.
class StreamListener(tweepy.StreamListener):
    
    def __init__(self):
        super().__init__()
        self.tweet_count=0
        self.TWEET_LIMIT = 9 
    def on_status(self, status):
        #self.tweet_count=0
        #self.TWEET_LIMIT = limit
        print(status.id_str)
        # if "retweeted_status" attribute exists, flag this tweet as a retweet.
        is_retweet = hasattr(status, "retweeted_status")

        # check if text has been truncated
        if hasattr(status,"extended_tweet"):
            text = status.extended_tweet["full_text"]
        else:
            text = status.text

        # check if this is a quote tweet.
        is_quote = hasattr(status, "quoted_status")
        quoted_text = ""
        if is_quote:
            # check if quoted tweet's text has been truncated before recording it
            if hasattr(status.quoted_status,"extended_tweet"):
                quoted_text = status.quoted_status.extended_tweet["full_text"]
            else:
                quoted_text = status.quoted_status.text

        # remove characters that might cause problems with csv encoding
        remove_characters = [",","\n"]
        for c in remove_characters:
            text.replace(c," ")
            quoted_text.replace(c, " ")

        with open("fuck.txt", "a", encoding='utf-8') as f:
            f.write("%s\n" % (text))
        self.tweet_count += 1
        #return self.tweet_count <= self.TWEET_LIMIT
        if self.tweet_count <= self.TWEET_LIMIT:
            return True
        else:
            # close file? nah
            # https://stackoverflow.com/questions/33498975/unable-to-stop-streaming-in-tweepy-after-one-minute
            # returining false to close stream
            return False
    def on_error(self, status_code):
        print("Encountered streaming error (", status_code, ")")
        sys.exit()

if __name__ == "__main__":
    # complete authorization and initialize API endpoint
    auth=tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)
    auth.set_access_token(keys.access_token,keys.access_token_secret)
    api=tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
    
  #  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  #  auth.set_access_token(access_key, access_secret)
   # api = tweepy.API(auth)

    # initialize stream
    streamListener = StreamListener()
    stream = tweepy.Stream(auth=api.auth, listener=streamListener,tweet_mode='extended')
    with open("fuck.txt", "w", encoding='utf-8') as f:
        f.write("date,user,is_retweet,is_quote,text,quoted_text\n")
    tags = ["Microsoft"]
    stream.filter(track=tags)

In [ ]:
tweet_text= Path('fuck.txt').read_text(errors='ignore')

Option Name	Option Short Code
URL   	p.OPT.URL
Mention	p.OPT.MENTION
Hashtag	p.OPT.HASHTAG
Reserved Words	p.OPT.RESERVED
Emoji	p.OPT.EMOJI
Smiley	p.OPT.SMILEY
Number	p.OPT.NUMBER

In [ ]:
p.set_options(p.OPT.URL, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER, p.OPT.HASHTAG,p.OPT.MENTION,p.OPT.NUMBER)

In [ ]:
tweet_text

In [ ]:
clean_tweet_text=p.clean(tweet_text)

In [ ]:
clean_tweet_text

In [ ]:
blob=TextBlob(clean_tweet_text)

In [ ]:
items=blob.word_counts.items()

In [ ]:
stop_words=stopwords.words('english')

In [ ]:
#elimiate stop words
items =[item for item in items if item[0] not in stop_words]

In [ ]:
from operator import itemgetter

In [ ]:
sorted_items=sorted(items, key=itemgetter(1), reverse=True)

In [ ]:
sorted_items[len(sorted_items)-1]

In [ ]:
top30=sorted_items[:10]

In [ ]:
sorted_items

In [ ]:
df=pd.DataFrame(top30, columns=['words','count'])

In [ ]:
df

In [ ]:
axes=df.plot.bar(x='words',y='count', legend=False)

In [ ]:
#conver df to tuple
tweet_tuples = [tuple(x) for x in df.values]

In [ ]:
Twitterwordcloud=WordCloud(width = 2000, height= 2000, prefer_horizontal=0.5, min_font_size=10, colormap='prism', background_color='white').generate_from_frequencies(dict(tweet_tuples))

In [ ]:
Twitter_wordcloud_1 = Twitterwordcloud.to_file('tw1.png')

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(Twitter_wordcloud_1)